In [1]:
import json
import numpy as np

In [2]:
def linear(x):
    return x

def derivative_linear(x):
    return np.ones(x.shape)

def relu(x):
    return np.maximum(0, x)

def derivative_relu(x):
    return np.where(x < 0, 0, 1)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def derivative_sigmoid(x):
    return sigmoid(x) * (1 - sigmoid(x))

def softmax(x):
    exp_x = np.exp(x)
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)

def derivative_softmax(x, isTarget):
    if isTarget:
        return softmax(x)
    else:
        return -(1-softmax(x))

In [3]:
def delta_output(activation_function, target, output, net):
  if activation_function == "linear":
      result = (target - output) * derivative_linear(net)
  elif activation_function == "relu":
      result = (target - output) * derivative_relu(net)
  elif activation_function == "sigmoid":
      result = (target - output) * derivative_sigmoid(net)
  elif activation_function == "softmax":
      result = -derivative_softmax(net, True)

  return result

def delta_hidden(activation_function, net, delta_output, weights):
  if activation_function == "linear":
      result = np.dot(delta_output, weights.T) * derivative_linear(net)
  elif activation_function == "relu":
      result = np.dot(delta_output, weights.T) * derivative_relu(net)
  elif activation_function == "sigmoid":
      result = np.dot(delta_output, weights.T) * derivative_sigmoid(net)
  elif activation_function == "softmax":
      result = -derivative_softmax(net, False)

  return result

def calculate_delta_weight(activation_function, learning_rate, target, output, net, weights, x, isOutputLayer):
    if isOutputLayer:
        return learning_rate * delta_output(activation_function, target, output, net) * x
    else:
        return learning_rate * delta_hidden(activation_function, net, delta_output, weights) * x

In [4]:
def loss_function_sse(target, output):
    print("loss function sse target: ", target)
    print("loss function sse output: ", output)
    
    e = 0.5 * np.sum((target - output) ** 2)
    return e

def loss_function_softmax(target, output):
    e = -np.sum(target * np.log(output))
    return e

In [5]:
def terminate_condition(stopped_by, max_iteration, error_threshold, iteration, error):
    if stopped_by == "max_iteration":
        return iteration >= max_iteration
    elif stopped_by == "error_threshold":
        return error <= error_threshold

def forward_propagation(model, input_data, weights):
    layers = model["layers"]
    output_layer = [input_data]

    for i, layer in enumerate(layers):
        activation_function = layer["activation_function"]
        weight_matrix = weights[i]
        bias = layer.get("bias", 1)

        input_activation = output_layer[-1]
        # print("Dimensi input_activation:", input_activation.shape)
        # print(input_activation)
        if len(input_activation.shape) == 1:
            input_activation_with_bias = np.insert(input_activation, 0, bias)
            input_activation_with_bias = input_activation_with_bias.reshape(1, -1)
        else:
            batch_size = input_activation.shape[0]
            bias_vector = np.ones((batch_size, 1)) * bias
            input_activation_with_bias = np.concatenate((bias_vector, input_activation), axis=1)
        # print("Dimensi input_activation_with_bias:", input_activation_with_bias.shape)
        # print(input_activation_with_bias)
        # print("Dimensi weight_matrix:", weight_matrix.shape)
        # print(weight_matrix)

        output_linear_combination = np.dot(input_activation_with_bias, weight_matrix)

        if activation_function == "linear":
            activation_result = linear(output_linear_combination)
        elif activation_function == "relu":
            activation_result = relu(output_linear_combination)
        elif activation_function == "sigmoid":
            activation_result = sigmoid(output_linear_combination)
        elif activation_function == "softmax":
            activation_result = softmax(output_linear_combination)

        output_layer.append(activation_result)

    return output_layer

def back_propagation(model, input, initial_weights, target_input, learning_rate, batch_size, stopped_by, max_iteration, error_threshold):

    epoch=0
    error = np.inf

    while not terminate_condition(stopped_by, max_iteration, error_threshold, epoch, error) :
        epoch += 1

        if error == np.inf:
            error = 0

        for batch in range(int(input.shape[0]/batch_size)):
            input_batch = input[batch*batch_size:(batch+1)*batch_size]
            target_batch = target_input[batch*batch_size:(batch+1)*batch_size]
            output_forward = forward_propagation(model, input_batch, initial_weights)
            
            delta_weight = [np.zeros(w.shape) for w in initial_weights]
            for id_data in range(batch_size):
                # Calculate error each output unit
                error_k = []
                id = id_data + batch_size * batch

                '''
                if (model["layers"][-1]["activation_function"] == "softmax"):
                    input_x = np.insert(output_forward[-2], 0, 1)
                    all_net = np.dot(input_x, initial_weights[-1].T)
                '''
                
                # Iterate through each output unit
                for i in range(len(output_forward[-1][id_data])):
                    activation_function = model["layers"][-1]["activation_function"]
                    output = output_forward[-1][id_data][i]
                    target = target_batch[id_data][i]

                    print("output:", output) 
                    print("output_forward :", output_forward)
                    print("initial_weights:", initial_weights[-1][:,i])
                    
                    input_x = output_forward[-2][id_data]
                    input_x = np.insert(input_x, 0, 1)
                    print("input x:", input_x)
                    
                    net = np.dot(input_x.T, initial_weights[-1][:,i])

                    weights = initial_weights[-1]

                    error_k_j = []
                    for j in range(input_x.shape[0]):
                        x = input_x[j]
                        
                        if (activation_function == "softmax"):
                            error_k_j.append(calculate_delta_weight(activation_function, learning_rate, target, output, all_net, weights, x, True))
                        else:
                            error_k_j.append(calculate_delta_weight(activation_function, learning_rate, target, output, net, weights, x, True))

                    error_k.append(error_k_j)
                        
                print("error_k Inside:", error_k)

                # Calculate error each hidden unit
                error_h = []
                
                # Iterate through each hidden layer
                for j in range(len(output_forward)-2, 1, -1):
                    error_h_j = []
                    activation_function = model["layers"][j]["activation_function"]

                    # Iterate through each hidden layer unit
                    for k in range(len(output_forward[j][id_data])):
                        output = output_forward[j][id_data][k]
                        target = target_batch[id_data][k]

                        print("output:", output) 
                        print("output_forward :", output_forward)
                        print("initial_weights:", initial_weights[j][:,k])
                        
                        input_x = output_forward[j-1][id_data]
                        input_x = np.insert(input_x, 0, 1)
                        print("input x:", input_x)
                        
                        net = np.dot(input_x.T, initial_weights[j][:,k])

                        weights = initial_weights[j]

                        error_h_j_k = []
                        for l in range(input_x.shape[0]):
                            x = input_x[l]
                            
                            if (activation_function == "softmax"):
                                error_h_j_k.append(calculate_delta_weight(activation_function, learning_rate, target, output, all_net, weights, x, False))
                            else:
                                error_h_j_k.append(calculate_delta_weight(activation_function, learning_rate, target, output, net, weights, x, False))

                        error_h_j.append(error_h_j_k)
                    error_h.append(error_h_j)

                # Update delta weight
                print ("error_k:", error_k)
                print ("delta_weight:", delta_weight)

                for j in range(len(initial_weights)-1, -1, -1):
                    if j == len(initial_weights)-1:
                        for k in range(len(initial_weights[j])):
                            for l in range(len(initial_weights[j][k])):
                                print("j:", j)
                                print("k:", k)
                                print("l:", l)
                                print("error_k[l][k]:", error_k[l][k])
                                print("delta_weight[j][k][l]:", delta_weight[j][k][l])
                                
                                delta_weight[j][k][l] += error_k[l][k]
                    else:
                        for k in range(len(initial_weights[j])):
                            for l in range(len(initial_weights[j][k])):
                                delta_weight[j][k][l] += error_h[j][k][l]

            # Update weight per batch
            # sum delta weight
            for i in range(len(initial_weights)):
                for j in range(len(initial_weights[i])):
                    for k in range(len(initial_weights[i][j])):
                        initial_weights[i][j][k] += delta_weight[i][j][k]

            print(" Weights: ", initial_weights)
            print("Delta Weights: ", delta_weight)

        # Calculate error
        for data in range(input.shape[0]):
            print("Data: ", data)
            print("target_input", target_input)
            print("input_data", input)
            
            output_forward = forward_propagation(model, input, initial_weights)
            print("output_forward", output_forward)

            activation_function = model["layers"][-1]["activation_function"]
            if activation_function == "softmax":
                error += loss_function_softmax(target_input[data], output_forward[-1][data])
            else:
                error += loss_function_sse(target_input[data], output_forward[-1][data])

    print("Final: ", initial_weights)
    return initial_weights
                

In [6]:
# Load JSON input
with open("test/backpropagation/mlp.json", "r") as file:
    data = json.load(file)

# Extract data
model = data["case"]["model"]
input_data = np.array(data["case"]["input"])
initial_weights = [np.array(layer_weights) for layer_weights in data["case"]["initial_weights"]]
target = np.array(data["case"]["target"])
learning_rate = data["case"]["learning_parameters"]["learning_rate"]
batch_size = data["case"]["learning_parameters"]["batch_size"]
max_iteration = data["case"]["learning_parameters"]["max_iteration"]
error_threshold = data["case"]["learning_parameters"]["error_threshold"]
stopped_by = data["expect"]["stopped_by"]
final_weights = [np.array(layer_weights) for layer_weights in data["expect"]["final_weights"]]

print("Initial Weights", initial_weights)

back_propagation(model, input_data, initial_weights, target, learning_rate, batch_size, stopped_by, max_iteration, error_threshold)

Initial Weights [array([[ 0.1,  0.3,  0.2],
       [ 0.4,  0.2, -0.7],
       [ 0.1, -0.8,  0.5]])]
output: 1.4000000000000001
output_forward : [array([[3., 1.],
       [1., 2.]]), array([[ 1.4,  0.1, -1.4],
       [ 0.7, -1.1,  0.5]])]
initial_weights: [0.1 0.4 0.1]
input x: [1. 3. 1.]
output: 0.09999999999999998
output_forward : [array([[3., 1.],
       [1., 2.]]), array([[ 1.4,  0.1, -1.4],
       [ 0.7, -1.1,  0.5]])]
initial_weights: [ 0.3  0.2 -0.8]
input x: [1. 3. 1.]
output: -1.4
output_forward : [array([[3., 1.],
       [1., 2.]]), array([[ 1.4,  0.1, -1.4],
       [ 0.7, -1.1,  0.5]])]
initial_weights: [ 0.2 -0.7  0.5]
input x: [1. 3. 1.]
error_k Inside: [[0.05999999999999999, 0.17999999999999997, 0.05999999999999999], [0.020000000000000004, 0.06000000000000001, 0.020000000000000004], [-0.05, -0.15000000000000002, -0.05]]
error_k: [[0.05999999999999999, 0.17999999999999997, 0.05999999999999999], [0.020000000000000004, 0.06000000000000001, 0.020000000000000004], [-0.05, -0.150

[array([[ 0.22,  0.36,  0.11],
        [ 0.64,  0.3 , -0.89],
        [ 0.28, -0.7 ,  0.37]])]

In [7]:
output_layer = forward_propagation(model, input_data, initial_weights)
print(output_layer)

[array([[3., 1.],
       [1., 2.]]), array([[ 2.42,  0.56, -2.19],
       [ 1.42, -0.74, -0.04]])]


In [8]:
# backward propagation with sklearn
from sklearn.neural_network import MLPClassifier

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier

# Load dataset
dataset = pd.read_csv("test/backpropagation/iris.csv")
learning_data = dataset.drop(columns=["Species"])
target_data = dataset["Species"]

mlp = MLPClassifier(hidden_layer_sizes=(5,9,3), max_iter=1000, activation='logistic', learning_rate='constant', learning_rate_init=0.01,)
mlp.fit(learning_data, target_data)

y_pred = mlp.predict(learning_data)
print("Predicted values:")
print(y_pred)
print("Actual values:")
print(target_data)
print("Accuracy:", mlp.score(learning_data, target_data))

Predicted values:
['Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa'
 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa'
 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa'
 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa'
 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa'
 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa'
 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa'
 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa'
 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa'
 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa'
 'Iris-versicolor' 'Iris-versicolor' 'Iris-versicolor' 'Iris-versicolor'
 'Iris-versicolor' 'Iris-versicolor' 'Iris-versicolor' 'Iris-versicolor'
 'Iris-versicolor' 'Iris-versicolor' 'Iris-versicolor' 'Iris-versicolor'
 'Iris-versicolor' 'Iris-versicolor' 'Iris-versicolor